# Testing the normal label formulation and the DL Enhancement / Lifted Formulation from https://link.springer.com/content/pdf/10.1007/s13676-012-0010-0.pdf

In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, readFile
import tsplib95
from imports import networkx as imp_nx
from network import min_tsp, gen_path_atsp, atsp_DT_formulation, tsp_callbacks
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt



def __write_solution(solution:list):
    print("writing solutions...")
    workbook = xlsxwriter.Workbook('BenchmarksTSP.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('F1', 'Objective Value', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark(paths:list):
    solutions = []
    i = 0
    
    for path in paths:
        tsp = tsplib95.load(path)
        
        G = tsp.get_graph()
        G = imp_nx.read(G)
        #m = gen_path_atsp.createGenModel(G,'min','metric')
        #m = atsp_DT_formulation.createGenModel(G, "min", "metric")
        m = tsp_callbacks.createGenModel(G, "min", "metric")
        m.params.MIPGap = 0.05
        m.params.TimeLimit = 1500
       

        m.optimize()

        solutions.append((i, len(G.G.edges()), len(G.G.nodes()), m.runTime, m.objVal, m.numConstrs, m.MIPGap))
        i += 1
    print(solutions)
    __write_solution(solutions)         

if __name__ == '__main__':
    

    paths = ["TSPTestInstances/bayg29.tsp",
             "TSPTestInstances/kroB150.tsp",
             "TSPTestInstances/bier127.tsp",
             ]
    
    runBenchmark(paths)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-03-23
Using license file C:\Program Files\Gurobi\gurobi.lic
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 87 rows, 870 columns and 1711 nonzeros
Model fingerprint: 0xb517974b
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 4e+02]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4671.0000000
Presolve removed 29 rows and 29 columns
Presolve time: 0.00s
Presolved: 58 rows, 841 columns, 1624 nonzeros
Variable types: 0 continuous, 841 integer (812 binary)

Root relaxation: objective 1.440000e+03, 67 itera

# Heuristic implementation. Using Nearest Neighbour to create a new Tour and 2 Opt to improve the tour

In [2]:
import sys, os
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, readFile
from imports import networkx as imp_nx
from network import tsp_heuristics as tsph
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt


nbr_cities = 150

path = r"TSPTestInstances/kroB150.xml"
distance = readFile.read_tsplib(path)
for i in range(nbr_cities):
    distance[i][i] = 10000 # Changing the diagonal to "infinity"
        
tour, length = tsph.nearNeighHeur(distance, nbr_cities)
tour, length = tsph.twoOptImprov(distance, tour, length, nbr_cities)
print(tour)
print(length)



32780.97760798702
32582.4068151069
32521.8260234623
32466.89909582187
32439.635218454445
32344.806780933497
32101.878466898714
32101.04394327812
32098.65167897882
31930.750646371656
31831.65741618169
31638.67196126317
31571.833600768237
31552.555947148343
31342.939182306756
31148.714648340057
30687.935330571585
30230.90949933509
30161.125106875617
30138.47707744038
30094.154807517676
30069.095070738324
29611.69783117996
29205.997065951917
29155.286703511934
29117.466415690484
29109.93590932114
28823.55145857978
28744.86601006973
28682.247946854055
28521.53103206373
28505.733467076403
28358.725418299233
28353.68572952103
28348.93640840375
28345.223449729216
28247.58662343944
28220.10305069716
28042.894598174484
28036.816183116083
27970.004339972395
27837.13276278211
27641.270115479987
27639.022527960078
27631.077466082854
27440.859637512724
27385.431485753703
27384.286806522592
27329.099372474862
27308.264896517736
27300.085977873146
27178.436964283414
27156.92990892518
27103.4249741472